Having just read the book, "The Man Who Solved The Markets: How Jim Simons Launched The Quant Revolution", I've become interested in the applications of pattern recognition in financial markets. While the book remains far more of a biography than an analysis of algorithmic trading strategies, it still mentions a few of the mathematical tools Simons employed to grow the Medallion Fund. Implementing most of these tools in practice lies far beyond my elementary coding abilities. That said, I have decided to try and get to grips with the basics of AI / pattern recognition by applying similar principles to the very simple game: Rock, Paper, Scissors.

This strategy in of itself is by no means advanced, however there are certain approaches we can take to develop it further. First, we need a way to combine the data from all the columns / patterns in deciding what hand to play next. To do that, we can convert the % figures back to integers. For each hand (R, P or S), we can then add up how many times each has come up in each pattern. Using a sum function, we can decide which hand to play. The output would look as follows. 

Although rock-paper-scissors (RPS) may seem like a trivial game, it actually involves the hard computational problem of temporal pattern recognition. This problem is fundamental to the fields of machine learning, artificial intelligence, and data compression.

To address this problem myself, I've decided to implement the following pattern recognition strategy.

Assume the list below shows the last 40 hands played by an opponent in a game of RPS.

['S','R','R','P','S','P','R','P','S','P','S','P','S','P','R','P','S','P','S','R','S','P','S','R','R','P','S','P','R','P','S','P','S','R','S','P','P','R','P','S'] ?

To judge what is most likely to be played in the 41st hand, we can start by looking at the 40th, Scissors, and see what the historical distribution of hands played following a Scissors is. From there, we can look at the combination of the 39th and 40th hand, Scissors preceded by Paper, and see what the distribution of hands played following that combination is. We can then train the code to repeat this process for all patterns up to a designated length of the total length of the list of hands already played (currently set at a quarter of total list length).

The table for the set of 40 hands above would look as follows. To select the recommended hand to next play, the program sums each of the N(Rock) / N(Paper) / N(Scissors) columns and takes the maximum to identify the hand most likely to be played next by the opponent. It recommendends the corresponding hand that would beat the opponent's one.

In [41]:
# output table from sample sequence

sample_sequence = ['S','R','R','P','S','P','R','P','S','P','S','P','S','P','R','P','S','P','S','R','S','P','S','R','R','P','S','P','R','P','S','P','S','R','S','P','P','R','P','S']
identifypattern(sample_sequence)

                           Pattern N(Rock) N(Paper) N(Scissors) N(Total)
1                              [S]       4        9           0       13
2                           [P, S]       3        7           0       10
3                        [R, P, S]       0        5           0        5
4                     [P, R, P, S]       0        3           0        3
5                  [P, P, R, P, S]       0        0           0        0
6               [S, P, P, R, P, S]       0        0           0        0
7            [R, S, P, P, R, P, S]       0        0           0        0
8         [S, R, S, P, P, R, P, S]       0        0           0        0
9      [P, S, R, S, P, P, R, P, S]       0        0           0        0
10  [S, P, S, R, S, P, P, R, P, S]       0        0           0        0
Recommended next hand: S


Here is the algorithm

In [1]:
import pandas as pd
import numpy as np
import random

In [40]:
def identifypattern(array):
    
    patterns = pd.DataFrame(columns = ['Pattern','N(Rock)','N(Paper)','N(Scissors)','N(Total)'])
    global outcome
    
    if len(array) == 1:
        outcome = random.choice(['R','P','S'])
        print('Random next hand: {}'.format(outcome))
    
    else:
        if len(array) % 4 == 0:
            n = int(len(array)/4)
        else: 
            n = int(round(len(array)/4))
      
        def countX(lst, x):
            count = 0
            for ele in lst:
                if (ele == x):
                    count = count + 1
            return count

        for i in range(0, n):

            # select pattern
            pattern = array[len(array)-i-1:len(array)]

            # find all instances where first element of pattern occurs in array
            possibles = [i for i, x in enumerate(array) if x == pattern[0]]

            # don't want one of the possibles to include the pattern itself
            possibles = [x for x in possibles if x < len(array)-len(pattern)]
            solns,hands = ([] for i in range(2))

            # now find all instances of the pattern
            for p in possibles:
                check = array[p:p+len(pattern)]
                if len(check) == len(pattern) and np.all(check == pattern):
                    solns.append(p)
            
            if len(solns) == 0: # i.e. no instances of this pattern have been previously recorded
                row = [pattern,0,0,0,0] # better to use np.NaN then 0?
                patterns.loc[len(patterns)] = row

            else: 
                # retrieve next hand played for all previous instances of pattern
                for sol in solns:
                    hands.append(array[sol+len(pattern)])

                hands_count = [pattern, countX(hands,'R'),countX(hands,'P'),countX(hands,'S'), (countX(hands,'R') + countX(hands,'P') + countX(hands,'S'))] 
                patterns.loc[len(patterns)] = hands_count 
        
        patterns.index = np.arange(1, len(patterns) + 1)
        print(patterns)

        outcome_dict = {'P': patterns['N(Rock)'].sum(), 'S': patterns['N(Paper)'].sum(), 'R': patterns['N(Scissors)'].sum()}
        outcome = max(outcome_dict, key=outcome_dict.get)
        print('Recommended next hand: {}'.format(outcome))

Here is the functionality for a human opponent to play against the program.

In [ ]:
user_array = np.array([], dtype = "object")

rounds = input("How many rounds do you want to play: ")

computer_score = 0
opponent_score = 0

for _ in range(int(rounds)):
    user_action = input("Enter a choice (R, P, S): ")
    user_array = np.append(user_array, user_action)
    print(user_array)
    markov1(user_array)
    if user_action == 'R' and outcome == 'P': 
        computer_score += 1
    if user_action == 'S' and outcome == 'R':
        computer_score += 1
    if user_action == 'P' and outcome == 'S':
        computer_score += 1
    if user_action == 'R' and outcome == 'S':
        opponent_score += 1
    if user_action == 'S' and outcome == 'P':
        opponent_score += 1
    if user_action == 'P' and outcome == 'R':
        opponent_score += 1
    print("Computer action: {}".format(outcome))
    print("Computer score: {}, Your score: {}".format(computer_score, opponent_score))

This is certainly a simple approach and there is much room for further improvement. Firstly, we could build in memory to the model so that recent historical patterns hold more weight than those idenitfied in the early stages of the game as an opponent may change his strategy. Second, we could devise a more advanced way of evaluating the table outputs, perhaps using significance tests for the data on each row / pattern. For now though, this similar pattern recognition approach should suffice in besting the vast majority of human opponents. 